In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Dec 21 02:13:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    25W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip3 install tokenizers wandb sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 33.6 MB/s 
     |████████████████████████████████| 1.9 MB 52.0 MB/s 
     |████████████████████████████████| 1.3 MB 67.2 MB/s 
     |████████████████████████████████| 182 kB 101.1 MB/s 
     |████████████████████████████████| 174 kB 92.2 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 173 kB 106.0 MB/s 
     |████████████████████████████████| 168 kB 104.6 MB/s 
     |████████████████████████████████| 168 kB 102.5 MB/s 
     |████████████████████████████████| 166 kB 94.7 MB/s 
     |████████████████████████████████| 166 kB 119.3 MB/s 
     |████████████████████████████████| 162 kB 112.6 MB/s 
     |████████████████████████████████| 162 kB 118.7 MB/s 
     |████████████████████████████████| 158 kB 113.6 MB/s 
     |████████████████████████████████| 157 kB 69.3 MB/s 
     |███████████████████

In [ ]:
!pip3 install transformers huggingface-hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 29.9 MB/s 
     |████████████████████████████████| 182 kB 97.5 MB/s 


In [ ]:
!pip3 install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 37.1 MB/s 
     |████████████████████████████████| 132 kB 103.1 MB/s 
     |████████████████████████████████| 212 kB 104.7 MB/s 
     |████████████████████████████████| 127 kB 105.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import os
os.chdir("drive/")
os.chdir('My Drive')
os.chdir('dacon')
os.chdir('sentence_classification')

In [ ]:
OUTPUT_DIR = './outputs_rnn/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# CFG

In [ ]:
# ====================================================
# CFG
# ====================================================
class CFG:
    wandb=True
    competition='sentence-classification'
    _wandb_kernel='bluehills'
    debug=False
    apex=True
    print_freq=100
    num_workers=4
    scheduler='cosine' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=50
    epochs=5
    encoder_lr=2e-5
    decoder_lr=2e-5
    min_lr=1e-6
    eps=1e-6
    betas=(0.9, 0.999)
    batch_size=10
    fc_dropout=0.1
    target_size=1 
    max_len=256
    weight_decay=0.01
    gradient_accumulation_steps=1
    max_grad_norm=1000
    seed=42
    n_fold=10
    trn_fold=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    train=True
    model="klue/roberta-large" # ["klue/roberta-large", "klue/bert-base"]
    use_rnn=True
    bidirectional=True
    use_fgm=True
    
if CFG.debug:
    CFG.epochs = 2
    CFG.trn_fold = [0]

In [ ]:
# ====================================================
# wandb
# ====================================================
if CFG.wandb:
    import wandb
    try:
        # from kaggle_secrets import UserSecretsClient
        # user_secrets = UserSecretsClient()
        # secret_value_0 = user_secrets.get_secret("wandb_api")
        # wandb.login(key=secret_value_0)
        print('login to wandb')
        wandb.login()
        anony = None
    except:
        anony = "must"
        print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')


    def class2dict(f):
        return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

    run = wandb.init(project='Transformer-Anatomy', 
                     name=CFG.model,
                     config=class2dict(CFG),
                     group=CFG.model,
                     job_type="train",
                     anonymous=anony)

login to wandb


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bluehills. Use `wandb login --relogin` to force relogin


# Library

In [ ]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
from math import sqrt
import shutil
import string
import pickle
import random
import joblib
import itertools
import logging
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedGroupKFold

import torch
print(f"torch.__version__: {torch.__version__}")
import torch.nn as nn
from torch.nn import Parameter, LSTM
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset
import torch.cuda.amp as amp

import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig, AutoModelForSequenceClassification
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from transformers import AutoTokenizer, AutoConfig, AutoModelForTokenClassification

from datasets import Dataset, load_metric

import matplotlib.font_manager as font_manager
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats

%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.__version__: 1.13.0+cu116
tokenizers.__version__: 0.13.2
transformers.__version__: 4.25.1
env: TOKENIZERS_PARALLELISM=true


# Utils

In [ ]:
# ====================================================
# Utils
# ====================================================

def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=CFG.seed)

In [ ]:
def compute_metrics(preds_type, preds_polarity, preds_tense, preds_certainty, labels_type, labels_polarity, labels_tense, labels_certainty):
    """ validation을 위한 metrics function """
    # labels = pred.label_ids
    # preds = pred.predictions.argmax(-1)
    # probs = pred.predictions

    # calculate accuracy using sklearn's function
    f1_score1 = f1_score( labels_type, preds_type, average='weighted' )
    f1_score2 = f1_score( labels_polarity, preds_polarity, average='weighted' )
    f1_score3 = f1_score( labels_tense, preds_tense, average='weighted' )
    f1_score4 = f1_score( labels_certainty, preds_certainty, average='weighted' )

    score = (f1_score1 + f1_score2 + f1_score3 + f1_score4) / 4

    return score

# Data Loading

In [ ]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")
submission = pd.read_csv('./sample_submission.csv')

In [ ]:
LABEL_MAPPING1 = {"사실형": 0, "추론형": 1, "대화형": 2, "예측형": 3} # 사실형, 추론형, 대화형, 예측형
LABEL_MAPPING2 = {"긍정": 0, "부정": 1, "미정": 2} # 긍정, 부정, 미정
LABEL_MAPPING3 = {"과거": 0, "현재": 1, "미래": 2} # 과거, 현재, 미래
LABEL_MAPPING4 = {"확실": 0, "불확실": 1} # 확실, 불확실

In [ ]:
train['label1']  = train['유형'].map(lambda x : LABEL_MAPPING1[x])
train['label2']  = train['극성'].map(lambda x : LABEL_MAPPING2[x])
train['label3']  = train['시제'].map(lambda x : LABEL_MAPPING3[x])
train['label4']  = train['확실성'].map(lambda x : LABEL_MAPPING4[x])

In [ ]:
display(train.head())
print(train.info())
print(train.shape)

,ID,문장,유형,극성,시제,확실성,label,label1,label2,label3,label4
0,TRAIN_00000,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실,0,0,1,0
1,TRAIN_00001,이어 ＂앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정＂이라며 ＂그 이전이...,사실형,긍정,과거,확실,사실형-긍정-과거-확실,0,0,0,0
2,TRAIN_00002,정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...,사실형,긍정,미래,확실,사실형-긍정-미래-확실,0,0,2,0
3,TRAIN_00003,"서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실,0,0,0,0
4,TRAIN_00004,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실,0,0,1,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16541 entries, 0 to 16540
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      16541 non-null  object
 1   문장      16541 non-null  object
 2   유형      16541 non-null  object
 3   극성      16541 non-null  object
 4   시제      16541 non-null  object
 5   확실성     16541 non-null  object
 6   label   16541 non-null  object
 7   label1  16541 non-null  int64 
 8   label2  16541 non-null  int64 
 9   label3  16541 non-null  int64 
 10  label4  16541 non-null  int64 
dtypes: int64(4), object(7)
memory usage: 1.4+ MB
None
(16541, 11)


In [ ]:
display(test.head())
print(test.info())
print(test.shape)

,ID,문장
0,TEST_0000,"장욱진의 ＇가족＇은 허물 없는 가족애를, 처음 공개되는 정약용의 ＇정효자전＇과 ＇정..."
1,TEST_0001,"조지 W 부시, 버락 오바마 전 대통령도 전쟁 위험 때문에 버린 카드다."
2,TEST_0002,지난해 1분기 128억원이었던 영업이익이 올해 1분기 505억원으로 급증했다.
3,TEST_0003,수상 작가와 맺으려던 계약서 내용 가운데 일부가 ＇독소 조항＇으로 해석돼 수정을 요...
4,TEST_0004,결국 최근 KDB산업은행은 대규모 손실 위기에 닥친 에어부산에 140억원 금융지원을...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7090 entries, 0 to 7089
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      7090 non-null   object
 1   문장      7090 non-null   object
dtypes: object(2)
memory usage: 110.9+ KB
None
(7090, 2)


In [ ]:
display(submission.head())
print(submission.info())
print(submission.shape)

,ID,label
0,TEST_0000,추론형-긍정-현재-확실
1,TEST_0001,추론형-긍정-현재-확실
2,TEST_0002,추론형-긍정-현재-확실
3,TEST_0003,추론형-긍정-현재-확실
4,TEST_0004,추론형-긍정-현재-확실


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7090 entries, 0 to 7089
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      7090 non-null   object
 1   label   7090 non-null   object
dtypes: object(2)
memory usage: 110.9+ KB
None
(7090, 2)


# Tokenizer

In [ ]:
# ====================================================
# tokenizer
# ====================================================
tokenizer = AutoTokenizer.from_pretrained(CFG.model, use_fast=True)
tokenizer.save_pretrained(OUTPUT_DIR+'tokenizer/')
CFG.tokenizer = tokenizer

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/752k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [ ]:
# max length
lengths_dict = {}

for text_col in ['ID', '문장']:
    lengths = []
    tk0 = tqdm(train[text_col].fillna("").values, total=len(train))
    for text in tk0:
        length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
        lengths.append(length)
    lengths_dict[text_col] = lengths

CFG.max_len = max(lengths_dict['ID']) + max(lengths_dict['문장']) + 2 # CLS + SEP

LOGGER.info(f"max_len: {CFG.max_len}")

  0%|          | 0/16541 [00:00<?, ?it/s]

  0%|          | 0/16541 [00:00<?, ?it/s]

max_len: 322
INFO:__main__:max_len: 322


# CV

In [ ]:
# ====================================================
# CV split
# ====================================================
Fold = GroupKFold(n_splits=CFG.n_fold)
groups = train['ID'].values

train['fold'] = train['ID']
train_fold = train['fold']

for n, (train_index, val_index) in enumerate(Fold.split(train, train['ID'], groups)):
    #train_fold.iloc[val_index] = int(n)
    train.loc[val_index, 'fold'] = int(n)
train['fold'] = train_fold
train['fold'] = train['fold'].astype(int)
display(train.groupby('fold').size())

fold
0    1655
1    1654
2    1654
3    1654
4    1654
5    1654
6    1654
7    1654
8    1654
9    1654
dtype: int64

In [ ]:
# !pip3 install -q iterative-stratification
# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

# dfx = pd.get_dummies(train, columns=["ID"]).groupby(["label1"], as_index=False).sum()
# cols = [c for c in dfx.columns if c.startswith("ID_") or c == "label1"]
# dfx = dfx[cols]

# mskf = MultilabelStratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=42)
# labels = [c for c in dfx.columns if c != "label1"]
# dfx_labels = dfx[labels]
# dfx["fold"] = -1

# for fold, (trn_, val_) in enumerate(mskf.split(dfx, dfx_labels)):
#     print(len(trn_), len(val_))
#     dfx.loc[val_, "fold"] = fold

# train = train.merge(dfx[["label1", "fold"]], on="label1", how="left")
# print(train.fold.value_counts())

In [ ]:
train.head()

,ID,문장,유형,극성,시제,확실성,label,label1,label2,label3,label4,fold
0,TRAIN_00000,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실,0,0,1,0,0
1,TRAIN_00001,이어 ＂앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정＂이라며 ＂그 이전이...,사실형,긍정,과거,확실,사실형-긍정-과거-확실,0,0,0,0,3
2,TRAIN_00002,정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...,사실형,긍정,미래,확실,사실형-긍정-미래-확실,0,0,2,0,2
3,TRAIN_00003,"서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실,0,0,0,0,1
4,TRAIN_00004,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실,0,0,1,0,0


# Dataset

In [ ]:
def prepare_input(cfg, text):
    # text = text[:cfg.essay_max_len]
    inputs = cfg.tokenizer(text,
            add_special_tokens=True,
            max_length=cfg.max_len,
            padding="max_length",
            return_attention_mask=True,
            return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs

In [ ]:
class Sentence_Dataset(torch.utils.data.Dataset):
    """ Dataset 구성을 위한 class."""
    def __init__(self, cfg, texts, labels1, labels2, labels3, labels4):
      self.cfg = cfg
      self.texts = texts
      self.labels1 = labels1
      self.labels2 = labels2
      self.labels3 = labels3
      self.labels4 = labels4

    def __getitem__(self, idx):
      input_ids = prepare_input(self.cfg, self.texts[idx])
      
      labels1 = torch.tensor(self.labels1[idx])
      labels2 = torch.tensor(self.labels2[idx])
      labels3 = torch.tensor(self.labels3[idx])
      labels4 = torch.tensor(self.labels4[idx])

      return input_ids, labels1, labels2, labels3, labels4

    def __len__(self):
      return len(self.labels1)

# Model

In [ ]:
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()

    def forward(self, last_hidden_state, attention_mask):
    # def forward(self, last_hidden_state):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        # sum_embeddings = torch.sum(last_hidden_state, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

In [ ]:
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False, use_rnn=False, bidirectional=True, num_of_classes=72):
        super().__init__()
        self.cfg = cfg
        self.use_rnn = use_rnn

        self.my_hidden_dim = 256

        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        dim = self.config.hidden_size
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(dim, self.my_hidden_dim)
        self._init_weights(self.fc)

        self.head_dropout = nn.Dropout(cfg.fc_dropout)
        self.type_clf = nn.Linear(self.my_hidden_dim,4)
        self.polarity_clf = nn.Linear(self.my_hidden_dim,3)
        self.tense_clf = nn.Linear(self.my_hidden_dim,3)
        self.certainty_clf = nn.Linear(self.my_hidden_dim,2)

        # check if rnn is used
        if self.use_rnn:
            self.bidirectional = bidirectional
            self.pooling = MeanPooling()
            self.seq_encoder = LSTM(dim, int(dim / 2), 1, dropout=0.1, bidirectional=self.bidirectional, batch_first=True)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        outputs = outputs.last_hidden_state[:, 0, :]
        outputs = torch.squeeze(outputs)
        # last_hidden_states = outputs[0]
        return outputs
    
    def encode(self, inputs):
        outputs = self.model(**inputs)
        outputs = outputs.last_hidden_state
        outputs, _ = self.seq_encoder(outputs)
        return outputs


    def forward(self, inputs):
        if self.use_rnn:
            features = self.encode(inputs)
            features = self.pooling(features, inputs['attention_mask'])
        else:
            features = self.feature(inputs)
        output = self.fc(self.fc_dropout(features))
        output = self.head_dropout(output)

        output_type = self.type_clf(output)
        output_polarity = self.polarity_clf(output)
        output_tense = self.tense_clf(output)
        output_certainty = self.certainty_clf(output)

        return (output_type, output_polarity, output_tense, output_certainty)

In [ ]:
from torch.nn.modules.loss import _WeightedLoss


class SmoothCrossEntropyLoss(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    def k_one_hot(self, targets:torch.Tensor, n_classes:int, smoothing=0.0):
        with torch.no_grad():
            targets = torch.empty(size=(targets.size(0), n_classes),
                                  device=targets.device) \
                                  .fill_(smoothing /(n_classes-1)) \
                                  .scatter_(1, targets.data.unsqueeze(1), 1.-smoothing)
        return targets

    def reduce_loss(self, loss):
        return loss.mean() if self.reduction == 'mean' else loss.sum() if self.reduction == 'sum' else loss

    def forward(self, inputs, targets):
        assert 0 <= self.smoothing < 1

        targets = self.k_one_hot(targets, inputs.size(-1), self.smoothing)
        log_preds = F.log_softmax(inputs, -1)

        if self.weight is not None:
            log_preds = log_preds * self.weight.unsqueeze(0)

        return self.reduce_loss(-(targets * log_preds).sum(dim=-1))

In [ ]:
class FGM():
    def __init__(self, model):
        self.model = model
        self.backup = {}

    def attack(self, epsilon=1., emb_name='word_embeddings'):
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                self.backup[name] = param.data.clone()
                norm = torch.norm(param.grad)
                if norm != 0:
                    r_at = epsilon * param.grad / norm
                    param.data.add_(r_at)

    def restore(self, emb_name='word_embeddings'):
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                if name in self.backup:
                    param.data = self.backup[name]
            self.backup = {}


# Training

In [ ]:
# ====================================================
# Helper functions
# ====================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def train_fn(fold, train_loader, model, criterion_type, criterion_polarity, criterion_tense, criterion_certainty, optimizer, epoch, scheduler, device):
    model.train()
    fgm = FGM(model)
    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex)
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0

    for step, (inputs, labels1, labels2, labels3, labels4) in enumerate(train_loader):
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels1 = labels1.to(device)
        labels2 = labels2.to(device)
        labels3 = labels3.to(device)
        labels4 = labels4.to(device)
        batch_size = labels1.size(0)

        with torch.cuda.amp.autocast(enabled=CFG.apex):
            y_preds = model(inputs)
        y_pred_type, y_pred_polarity, y_pred_tense, y_pred_certainty = y_preds[0], y_preds[1], y_preds[2], y_preds[3]

        loss1 = criterion_type(y_pred_type, labels1)
        loss2 = criterion_polarity(y_pred_polarity, labels2)
        loss3 = criterion_tense(y_pred_tense, labels3)
        loss4 = criterion_certainty(y_pred_certainty, labels4)

        loss = loss1 * 0.25 + loss2 * 0.25 + loss3 * 0.25 + loss4 * 0.25

        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        
        # -------------------
        # FGM
        if CFG.use_fgm:
            fgm.attack()
            with torch.cuda.amp.autocast(enabled=CFG.apex):
                y_preds = model(inputs)
            y_pred_type, y_pred_polarity, y_pred_tense, y_pred_certainty = y_preds[0], y_preds[1], y_preds[2], y_preds[3]
            
            loss_adv1 = criterion_type(y_pred_type, labels1)
            loss_adv2 = criterion_polarity(y_pred_polarity, labels2)
            loss_adv3 = criterion_tense(y_pred_tense, labels3)
            loss_adv4 = criterion_certainty(y_pred_certainty, labels4)

            loss_adv = loss_adv1 * 0.25 + loss_adv2 * 0.25 + loss_adv3 * 0.25 + loss_adv4 * 0.25

            loss_adv.backward()
            fgm.restore()
        # -------------------
        
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
            if CFG.batch_scheduler:
                scheduler.step()
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.8f}  '
                  .format(epoch+1, step, len(train_loader), 
                          remain=timeSince(start, float(step+1)/len(train_loader)),
                          loss=losses,
                          grad_norm=grad_norm,
                          lr=scheduler.get_lr()[0])) 
        if CFG.wandb:
            wandb.log({f"[fold{fold}] loss": losses.val,
                       f"[fold{fold}] lr": scheduler.get_lr()[0]})
        torch.cuda.empty_cache()
        gc.collect()
    return losses.avg


def append_pred_to_list(preds, y_preds):
    pred_np = y_preds.sigmoid().to('cpu').numpy()
    if len(pred_np.shape) == 1:
        pred_np = np.array([pred_np.tolist()])
    preds.append(pred_np)

def valid_fn(valid_loader, model, criterion_type, criterion_polarity, criterion_tense, criterion_certainty, device):
    losses = AverageMeter()
    model.eval()
    
    preds_type, preds_polarity, preds_tense, preds_certainty = [], [], [], []
    
    start = end = time.time()
    for step, (inputs, labels1, labels2, labels3, labels4) in enumerate(valid_loader):
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels1 = labels1.to(device)
        labels2 = labels2.to(device)
        labels3 = labels3.to(device)
        labels4 = labels4.to(device)
        batch_size = labels1.size(0)
        
        with torch.no_grad():
            y_preds = model(inputs)
        y_pred_type, y_pred_polarity, y_pred_tense, y_pred_certainty = y_preds[0], y_preds[1], y_preds[2], y_preds[3]

        loss1 = criterion_type(y_pred_type, labels1)
        loss2 = criterion_polarity(y_pred_polarity, labels2)
        loss3 = criterion_tense(y_pred_tense, labels3)
        loss4 = criterion_certainty(y_pred_certainty, labels4)

        loss = loss1 * 0.25 + loss2 * 0.25 + loss3 * 0.25 + loss4 * 0.25
        
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        
        append_pred_to_list(preds_type, y_pred_type)
        append_pred_to_list(preds_polarity, y_pred_polarity)
        append_pred_to_list(preds_tense, y_pred_tense)
        append_pred_to_list(preds_certainty, y_pred_certainty)
        
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(step, len(valid_loader),
                          loss=losses,
                          remain=timeSince(start, float(step+1)/len(valid_loader))))
    predictions_type = np.concatenate(preds_type)
    predictions_polarity = np.concatenate(preds_polarity)
    predictions_tense = np.concatenate(preds_tense)
    predictions_certainty = np.concatenate(preds_certainty)

    return losses.avg, predictions_type, predictions_polarity, predictions_tense, predictions_certainty

In [ ]:
# ====================================================
# train loop
# ====================================================
def train_loop(folds, fold):
    
    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    train_folds = folds[folds['fold'] != fold].reset_index(drop=True)
    valid_folds = folds[folds['fold'] == fold].reset_index(drop=True)

    train_labels1 = train_folds['label1'].values
    train_labels2 = train_folds['label2'].values
    train_labels3 = train_folds['label3'].values
    train_labels4 = train_folds['label4'].values

    valid_labels1 = valid_folds['label1'].values
    valid_labels2 = valid_folds['label2'].values
    valid_labels3 = valid_folds['label3'].values
    valid_labels4 = valid_folds['label4'].values

    train_texts = train_folds['문장'].values
    valid_texts = valid_folds['문장'].values
    
    train_dataset = Sentence_Dataset(CFG, train_texts, train_labels1, train_labels2, train_labels3, train_labels4)
    valid_dataset = Sentence_Dataset(CFG, valid_texts, valid_labels1, valid_labels2, valid_labels3, valid_labels4)

    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)


    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomModel(CFG, config_path=None, pretrained=True, use_rnn=CFG.use_rnn, bidirectional=CFG.bidirectional)
    torch.save(model.config, OUTPUT_DIR+'config.pth')
    model.to(device)
    
    def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.0):
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_parameters = [
            {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': weight_decay},
            {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': 0.0},
            {'params': [p for n, p in model.named_parameters() if "model" not in n],
             'lr': decoder_lr, 'weight_decay': 0.0}
        ]
        return optimizer_parameters

    optimizer_parameters = get_optimizer_params(model,
                                                encoder_lr=CFG.encoder_lr, 
                                                decoder_lr=CFG.decoder_lr,
                                                weight_decay=CFG.weight_decay)
    optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)
    
    # ====================================================
    # scheduler
    # ====================================================
    def get_scheduler(cfg, optimizer, num_train_steps):
        if cfg.scheduler=='linear':
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
            )
        elif cfg.scheduler=='cosine':
            scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
            )
        return scheduler
    
    num_train_steps = int(len(train_folds) / CFG.batch_size * CFG.epochs)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)

    # ====================================================
    # loop
    # ====================================================
    # criterion = nn.BCEWithLogitsLoss(reduction="mean")
    criterion_type = SmoothCrossEntropyLoss(smoothing=0.1)
    criterion_polarity = SmoothCrossEntropyLoss(smoothing=0.1)
    criterion_tense = SmoothCrossEntropyLoss(smoothing=0.1)
    criterion_certainty = SmoothCrossEntropyLoss(smoothing=0.1)
    
    best_score = 0.

    for epoch in range(CFG.epochs):
        start_time = time.time()

        # train
        avg_loss = train_fn(fold, train_loader, model, criterion_type, criterion_polarity, criterion_tense, criterion_certainty, optimizer, epoch, scheduler, device)
        
        # eval
        avg_val_loss, predictions1, predictions2, predictions3, predictions4 = valid_fn(valid_loader, model, criterion_type, criterion_polarity, criterion_tense, criterion_certainty, device)
        
        # scoring
        y_predictions1 = predictions1.argmax(-1)
        y_predictions2 = predictions2.argmax(-1)
        y_predictions3 = predictions3.argmax(-1)
        y_predictions4 = predictions4.argmax(-1)

        score = compute_metrics(y_predictions1, y_predictions2, y_predictions3, y_predictions4, valid_labels1, valid_labels2, valid_labels3, valid_labels4)
        # score, auprc, accuracy = metric_result['micro f1 score'], metric_result['auprc'], metric_result['accuracy']

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - F1-Score: {score:.4f}')
        if CFG.wandb:
            wandb.log({f"[fold{fold}] epoch": epoch+1, 
                       f"[fold{fold}] avg_train_loss": avg_loss, 
                       f"[fold{fold}] avg_val_loss": avg_val_loss,
                       f"[fold{fold}] score": score})
        
        if best_score < score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'predictions1': predictions1,
                        'predictions2': predictions2,
                        'predictions3': predictions3,
                        'predictions4': predictions4},
                        OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth")

        torch.cuda.empty_cache()
        gc.collect()

    torch.cuda.empty_cache()
    gc.collect()
    
    return best_score

In [ ]:
if __name__ == '__main__':
    if CFG.train:
        scores = []
        for fold in range(CFG.n_fold):
            torch.cuda.empty_cache()
            if fold in CFG.trn_fold:
                torch.cuda.empty_cache()
                score = train_loop(train, fold)
                LOGGER.info(f"========== fold: {fold} result ==========")
                LOGGER.info(f'Score: {score:<.4f}')
                scores.append(score)
        LOGGER.info(f"========== CV ==========")
        score = np.mean(scores)
        LOGGER.info(f'Score: {score:<.4f}')
        
    if CFG.wandb:
        wandb.finish()

========== fold: 0 training ==========
INFO:__main__:========== fold: 0 training ==========


Downloading:   0%|          | 0.00/547 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it f

Epoch: [1][0/1488] Elapsed 0m 2s (remain 62m 57s) Loss: 1.0562(1.0562) Grad: 78798.1797  LR: 0.00000040  
Epoch: [1][100/1488] Elapsed 2m 41s (remain 36m 53s) Loss: 0.5859(0.8109) Grad: 69199.3203  LR: 0.00002000  
Epoch: [1][200/1488] Elapsed 5m 23s (remain 34m 32s) Loss: 0.6784(0.7112) Grad: 54327.7891  LR: 0.00001998  
Epoch: [1][300/1488] Elapsed 8m 6s (remain 31m 58s) Loss: 0.5837(0.6685) Grad: 514093.8438  LR: 0.00001994  
Epoch: [1][400/1488] Elapsed 10m 49s (remain 29m 19s) Loss: 0.5138(0.6388) Grad: 24117.7285  LR: 0.00001989  
Epoch: [1][500/1488] Elapsed 13m 31s (remain 26m 39s) Loss: 0.6038(0.6214) Grad: 85087.2656  LR: 0.00001982  
Epoch: [1][600/1488] Elapsed 16m 14s (remain 23m 58s) Loss: 0.5649(0.6083) Grad: 44193.7305  LR: 0.00001973  
Epoch: [1][700/1488] Elapsed 18m 56s (remain 21m 16s) Loss: 0.5371(0.5971) Grad: 57555.0547  LR: 0.00001962  
Epoch: [1][800/1488] Elapsed 21m 39s (remain 18m 34s) Loss: 0.5985(0.5875) Grad: 69798.9297  LR: 0.00001950  
Epoch: [1][900/14

...

EVAL: [165/166] Elapsed 1m 48s (remain 0m 0s) Loss: 0.7057(0.5090) 
Epoch: [2][0/1488] Elapsed 0m 1s (remain 39m 40s) Loss: 0.5039(0.5039) Grad: 79896.7422  LR: 0.00001819  
Epoch: [2][100/1488] Elapsed 2m 43s (remain 37m 30s) Loss: 0.4459(0.5038) Grad: 101867.5938  LR: 0.00001794  
Epoch: [2][200/1488] Elapsed 5m 25s (remain 34m 47s) Loss: 0.4447(0.4999) Grad: 122180.2734  LR: 0.00001767  
Epoch: [2][300/1488] Elapsed 8m 8s (remain 32m 4s) Loss: 0.6524(0.4988) Grad: 120794.4375  LR: 0.00001739  
Epoch: [2][400/1488] Elapsed 10m 50s (remain 29m 22s) Loss: 0.4817(0.4965) Grad: 220585.6406  LR: 0.00001710  
Epoch: [2][500/1488] Elapsed 13m 32s (remain 26m 40s) Loss: 0.5047(0.4951) Grad: 119435.4141  LR: 0.00001679  
Epoch: [2][600/1488] Elapsed 16m 14s (remain 23m 58s) Loss: 0.5476(0.4950) Grad: 88891.7500  LR: 0.00001648  
Epoch: [2][700/1488] Elapsed 18m 56s (remain 21m 16s) Loss: 0.4676(0.4943) Grad: 89865.1719  LR: 0.00001615  
Epoch: [2][800/1488] Elapsed 21m 38s (remain 18m 34s) Lo

Epoch 2 - avg_train_loss: 0.4894  avg_val_loss: 0.4975  time: 2521s
INFO:__main__:Epoch 2 - avg_train_loss: 0.4894  avg_val_loss: 0.4975  time: 2521s
Epoch 2 - F1-Score: 0.9235
INFO:__main__:Epoch 2 - F1-Score: 0.9235
Epoch 2 - Save Best Score: 0.9235 Model
INFO:__main__:Epoch 2 - Save Best Score: 0.9235 Model


EVAL: [165/166] Elapsed 1m 48s (remain 0m 0s) Loss: 0.6976(0.4975) 
Epoch: [3][0/1488] Elapsed 0m 1s (remain 39m 59s) Loss: 0.4022(0.4022) Grad: 36033.9922  LR: 0.00001321  
Epoch: [3][100/1488] Elapsed 2m 43s (remain 37m 30s) Loss: 0.4129(0.4693) Grad: 71710.0469  LR: 0.00001281  
Epoch: [3][200/1488] Elapsed 5m 25s (remain 34m 47s) Loss: 0.4701(0.4661) Grad: 68792.6797  LR: 0.00001240  
Epoch: [3][300/1488] Elapsed 8m 8s (remain 32m 4s) Loss: 0.4785(0.4669) Grad: 68154.4141  LR: 0.00001198  
Epoch: [3][400/1488] Elapsed 10m 50s (remain 29m 22s) Loss: 0.4413(0.4681) Grad: 84636.8438  LR: 0.00001156  
Epoch: [3][500/1488] Elapsed 13m 32s (remain 26m 40s) Loss: 0.4723(0.4689) Grad: 96580.9375  LR: 0.00001114  
Epoch: [3][600/1488] Elapsed 16m 14s (remain 23m 58s) Loss: 0.4971(0.4704) Grad: 98542.6875  LR: 0.00001072  
Epoch: [3][700/1488] Elapsed 18m 56s (remain 21m 15s) Loss: 0.4711(0.4693) Grad: 84959.3828  LR: 0.00001030  
Epoch: [3][800/1488] Elapsed 21m 38s (remain 18m 33s) Loss: 0

Epoch 3 - avg_train_loss: 0.4686  avg_val_loss: 0.5041  time: 2521s
INFO:__main__:Epoch 3 - avg_train_loss: 0.4686  avg_val_loss: 0.5041  time: 2521s
Epoch 3 - F1-Score: 0.9267
INFO:__main__:Epoch 3 - F1-Score: 0.9267
Epoch 3 - Save Best Score: 0.9267 Model
INFO:__main__:Epoch 3 - Save Best Score: 0.9267 Model


EVAL: [165/166] Elapsed 1m 48s (remain 0m 0s) Loss: 0.7033(0.5041) 
Epoch: [4][0/1488] Elapsed 0m 1s (remain 39m 33s) Loss: 0.4437(0.4437) Grad: 111314.8906  LR: 0.00000699  
Epoch: [4][100/1488] Elapsed 2m 43s (remain 37m 28s) Loss: 0.4521(0.4495) Grad: 94151.2891  LR: 0.00000659  
Epoch: [4][200/1488] Elapsed 5m 25s (remain 34m 45s) Loss: 0.4068(0.4488) Grad: 41769.5312  LR: 0.00000620  
Epoch: [4][300/1488] Elapsed 8m 7s (remain 32m 3s) Loss: 0.4433(0.4507) Grad: 94555.1016  LR: 0.00000581  
Epoch: [4][400/1488] Elapsed 10m 49s (remain 29m 21s) Loss: 0.4781(0.4497) Grad: 228466.9375  LR: 0.00000542  
Epoch: [4][500/1488] Elapsed 13m 31s (remain 26m 39s) Loss: 0.4311(0.4500) Grad: 64714.4609  LR: 0.00000505  
Epoch: [4][600/1488] Elapsed 16m 13s (remain 23m 57s) Loss: 0.4367(0.4505) Grad: 126378.1172  LR: 0.00000469  
Epoch: [4][700/1488] Elapsed 18m 55s (remain 21m 14s) Loss: 0.5109(0.4505) Grad: 72648.7344  LR: 0.00000433  
Epoch: [4][800/1488] Elapsed 21m 37s (remain 18m 32s) Loss